In [ ]:
# !pip install opencv-python transformers accelerate
from diffusers import StableDiffusionPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import numpy as np
import torch

import cv2
from PIL import Image
from controlnet_aux import MidasDetector,OpenposeDetector,HEDdetector
# pipe = StableDiffusionPipeline.from_pretrained(
#     "runwayml/stable-diffusion-v1-5"
# )

# speed up diffusion process with faster scheduler and memory optimization

midas = MidasDetector.from_pretrained("lllyasviel/Annotators")
openpose = OpenposeDetector.from_pretrained("lllyasviel/Annotators")
hed = HEDdetector.from_pretrained("lllyasviel/Annotators")

generator = torch.manual_seed(100)
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
controlnet1= ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose")

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet1
)
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
from controlnet_naive import ControlNetModel
statedictpipe=pipe.controlnet.state_dict()
new_unet=ControlNetModel()
new_unet.load_state_dict(statedictpipe)
pipe.controlnet=new_unet
edge_image=load_image('./spider5.jpg').convert("RGB")
image=openpose(edge_image)
image=Image.fromarray(np.array(image))
# image = cv2.Canny(edge_image, 100,200)
# image = image[:, :, None]
# image = np.concatenate([image, image, image], axis=2)
# image = Image.fromarray(image)
# image.resize((512,512)).save('canny.png')
pipe.to("cuda")


In [ ]:
edge_image

In [ ]:
# from controlnet_naive import ControlNetModel
# statedictpipe=pipe.controlnet.state_dict()
# new_unet=ControlNetModel()

In [ ]:
edge_image=load_image('./luffy.jpg').convert("RGB")
image=openpose(edge_image)
image=Image.fromarray(np.array(image))
image1=image.resize((512,512))
image1.save('luffy_pose.png')

In [ ]:
# new_unet.load_state_dict(statedictpipe)
# pipe.unet=new_unet

In [ ]:
import ptp_utils_naive_control as ptp_utils
def run_and_display(pipe,prompts, negative_prompts, latent=None, run_baseline=False, generator=None,  control=None , results_dir='results'):

    images, x_t, all_list_encoder= ptp_utils.text2image_ldm_stable(pipe, prompts, negative_prompts, latent=latent, num_inference_steps=50, guidance_scale=7.5, generator=generator,control=control, low_resource=False)

    # ptp_utils.view_images(images,results_dir=results_dir)
    return images, x_t, all_list_encoder

In [ ]:
prompts=["Spiderman looking at a city, cyberpunk,hd , best quality, high quality"]
negative_prompts=["Bad quality, bad anatomy"]

In [ ]:
pipe = pipe.to("cuda")
g_cpu = torch.Generator().manual_seed(100)
latent = torch.randn(
                (1, 4,512 // 8, 512 // 8),
                generator=g_cpu,
            )
a,b,c=run_and_display(pipe,prompts, negative_prompts, latent=latent, run_baseline=False, generator=None, control=image, results_dir='results')
ptp_utils.view_images(a)

In [ ]:
a.save('luffy-pose.png')
stop

In [ ]:
import os

# import nump
from PIL import Image
import matplotlib.pyplot as plt

def save_image(imagea, loc):
    # if image.shape[1]>=64:
        imagea=imagea-imagea.min()
        imagea=imagea/imagea.max()
        imagea = imagea[0].cpu().cpu().numpy()
        # image=(image-np.min(image))(np.max(image)-np.min(image))
        imagea = (imagea * 255).astype(np.uint8)
        imagea=Image.fromarray(imagea)
        imagea=imagea.resize((256,256))
        image_array = np.array(imagea)

        # Create a heatmap from the array
        plt.imshow(image_array, cmap='viridis')

        # Remove axes and save the heatmap
        plt.axis('off')
        plt.savefig(loc, bbox_inches='tight', pad_inches=0)
        return imagea
    # return None
    
print(len(c))
count=0
for list1 in c:
    count=count+1
    # print(list1)
    for list2 in list1:
        # print(list2.shape,count)
        # print(list2)
        if os.path.exists('spider_pose')==False:
             os.makedirs('spider_pose')
        if image is not None:
            dirname=len(os.listdir('./spider_pose'))
        loc = os.path.join('./spider_pose', str(dirname)+'.png')
        image=save_image(list2,loc)


In [ ]:
image